# Code Broker: Multi-Agent Code Assessment System

This notebook implements a sophisticated multi-agent system for comprehensive code analysis using Google's ADK (Agent Development Kit).

## Features
- 🔍 **Code Description Generation**: Analyzes code structure and functionality
- ✅ **Correctness Assessment**: Evaluates functionality, security, and efficiency
- 🎨 **Style Assessment**: Checks readability, maintainability, and best practices
- 💡 **Improvement Recommendations**: Suggests actionable enhancements
- 📊 **Comprehensive Reporting**: Compiles all assessments into a professional report

## Requirements
- Google ADK
- Python 3.10+
- pylint (for code linting)
- Git (for repository cloning)

## 1️⃣ Imports and Dependencies

Import all necessary libraries for the multi-agent system:
- **Google ADK**: Agent framework components
- **asyncio**: Asynchronous execution
- **IPython**: Notebook display utilities

In [18]:
import os
import re
import shutil
import tempfile
import asyncio
import time
from pathlib import Path
from IPython.display import display, Markdown
from google.adk.models.google_llm import Gemini
from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.apps.app import App, ResumabilityConfig
from google.genai import types
from dotenv import load_dotenv

## 2️⃣ Environment Initialization

This function loads API credentials from your `.env` file:
- `GOOGLE_API_KEY`: Required for Gemini model access
- `GITHUB_TOKEN`: Optional, for private repository access

**Note**: Create a `.env` file in your project root with these variables.

In [19]:
async def initialize_adk_model():
    """
    Initializes the Google ADK environment with required API credentials.
    
    Raises:
        ValueError: If GOOGLE_API_KEY or GITHUB_TOKEN are not set
        
    Returns:
        dict: Configuration dictionary with API credentials
    """
    load_dotenv()
    api_key = os.getenv("GOOGLE_API_KEY")
    github_token = os.getenv("GITHUB_TOKEN")
    
    os.environ["GOOGLE_API_KEY"] = api_key
    os.environ["GITHUB_TOKEN"] = github_token
    
    if not api_key:
        raise ValueError("GOOGLE_API_KEY environment variable not set.")
    if not github_token:
        raise ValueError("GITHUB_TOKEN environment variable not set.")
    
    return {"api_key": api_key, "github_token": github_token}

## 3️⃣ API Retry Configuration

Configure exponential backoff retry behavior to handle:
- Rate limiting (HTTP 429)
- Transient server errors (HTTP 500, 503, 504)
- Network timeouts

In [20]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Exponential backoff multiplier
    initial_delay=1,  # Initial delay in seconds
    http_status_codes=[429, 500, 503, 504],  # Retry on these errors
)

## 4️⃣ File and Repository Utility Functions

These tools enable agents to:
- Read individual files or entire directories
- Clone and analyze GitHub repositories
- Run pylint for code quality assessment
- Clean up temporary directories

In [21]:
def read_file(file_path: str) -> str:
    """Reads and returns file content with error handling."""
    if not os.path.exists(file_path):
        return f"Error: File not found at {file_path}"
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"


def read_directory_files(directory_path: str) -> dict[str, str]:
    """Recursively reads all files in a directory, excluding .git."""
    if not os.path.isdir(directory_path):
        return {"error": f"Error: Directory not found at {directory_path}"}
    
    file_contents = {}
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if '.git' in file_path.split(os.sep):
                continue
            file_contents[file_path] = read_file(file_path)
    return file_contents


async def read_github_repository(repo_url: str) -> dict:
    """Clones a GitHub repository and reads all its files."""
    print(f"Debug: Cloning repository: {repo_url}")
    
    if not re.match(r"https://github\.com/([^/]+)/([^/]+)", repo_url):
        return {"error": "Invalid GitHub repository URL provided."}

    temp_dir = tempfile.mkdtemp()
    try:
        command = f"git clone --depth 1 {repo_url} ."
        process = await asyncio.create_subprocess_shell(
            command,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
            cwd=temp_dir
        )
        stdout, stderr = await process.communicate()

        if process.returncode != 0:
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)
            return {"error": f"Failed to clone repository. Error: {stderr.decode()}"}

        file_contents = read_directory_files(temp_dir)
        return {"file_contents": file_contents, "temp_dir": temp_dir}
    except Exception as e:
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        return {"error": f"An unexpected error occurred: {e}"}


def cleanup_temp_directory(directory_path: str) -> dict:
    """Safely removes a temporary directory and its contents."""
    if not os.path.isdir(directory_path):
        return {"error": f"Error: Directory not found at {directory_path}"}
    try:
        shutil.rmtree(directory_path)
        return {"success": f"Successfully removed directory: {directory_path}"}
    except Exception as e:
        return {"error": f"Error removing directory {directory_path}: {e}"}


async def get_linting_score(path: str) -> dict:
    """Runs pylint and returns a quality score (0-100%)."""
    if not os.path.exists(path):
        return {"error": f"Error: Path not found at {path}"}
    
    if os.path.isfile(path) and not path.endswith('.py'):
        return {"error": "Error: Not a Python file."}

    try:
        absolute_path = os.path.abspath(path)
        command = f"pylint {absolute_path}"
        
        process = await asyncio.create_subprocess_shell(
            command,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE
        )
        stdout, stderr = await process.communicate()
        pylint_output = stdout.decode()
        pylint_error = stderr.decode()

        if process.returncode != 0 and "No such file or directory" in pylint_error:
            return {"error": "Pylint not installed. Install with: pip install pylint"}
        
        match = re.search(r"Your code has been rated at ([-+]?\d*\.\d+|\d+)/10", pylint_output)
        if match:
            score_out_of_10 = float(match.group(1))
            return {"linting_score": score_out_of_10 * 10}
        else:
            return {
                "linting_score": 0.0,
                "message": "Pylint ran but no score found",
                "pylint_output": pylint_output[:500]
            }
    except Exception as e:
        return {"error": f"Linting error: {e}"}

## 5️⃣ Specialized Assessment Agents

### Agent Architecture:
1. **Correctness Assessor**: Evaluates functionality, security, efficiency, and tests
2. **Style Assessor**: Checks readability, maintainability, and best practices
3. **Description Generator**: Creates comprehensive code documentation
4. **Improvement Recommender**: Suggests actionable enhancements
5. **Report Generator**: Orchestrates all agents and compiles final report

Each agent has access to specialized tools for code analysis.

In [22]:
# AGENT 1: Correctness Assessor
correctness_assessor = LlmAgent(
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    name="correctness_assessor",
    description="Evaluates code functionality, security, and efficiency",
    instruction="""
        You are a code correctness expert. Evaluate the code on:
        1. **Functionality**: Error handling, edge cases, requirement delivery
        2. **Security**: Vulnerabilities, exploits, insecure practices
        3. **Resource Efficiency**: CPU, memory, resource management
        4. **Test Coverage**: Presence and quality of tests

        Provide analysis (2-3 sentences) and score (0-100%) for each.
        Present in structured Markdown with overall score.
        """,
    tools=[
        FunctionTool(read_file),
        FunctionTool(read_directory_files),
        FunctionTool(read_github_repository),
        FunctionTool(get_linting_score),
        FunctionTool(cleanup_temp_directory),
    ],
)

# AGENT 2: Style Assessor
style_assessor = LlmAgent(
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    name="style_assessor",
    description="Evaluates code style and maintainability",
    instruction="""
        You are a code style expert. Evaluate on:
        - **Readability**: Naming, formatting, comments
        - **Maintainability**: Structure, modularity, duplication
        - **Linting**: Use get_linting_score tool
        - **Best Practices**: README, .gitignore, requirements.txt

        Provide analysis (2-3 sentences) and score (0-100%) for each.
        Present in Markdown. Do not output raw file contents.
        """,
    tools=[
        FunctionTool(read_file),
        FunctionTool(read_directory_files),
        FunctionTool(read_github_repository),
        FunctionTool(get_linting_score),
        FunctionTool(cleanup_temp_directory),
    ],
)

# AGENT 3: Description Generator
description_generator = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="description_generator",
    description="Generates comprehensive code descriptions",
    instruction="""
        You are an expert at code documentation. Analyze and describe:
        1. **Overview**: High-level purpose, architecture, languages/libraries
        2. **Component Breakdown**: Key files, classes, functions, their roles
        3. **Functionality Summary**: Concise one-line summary

        Use Markdown formatting. Do not include raw code.
        """,
    tools=[
        FunctionTool(read_file),
        FunctionTool(read_directory_files),
        FunctionTool(read_github_repository),
        FunctionTool(get_linting_score),
        FunctionTool(cleanup_temp_directory),
    ],
)

# AGENT 4: Improvement Recommender
improvement_recommender = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="improvement_recommender",
    tools=[
        AgentTool(description_generator),
        AgentTool(correctness_assessor),
        AgentTool(style_assessor)
    ],
    description="Suggests actionable code improvements",
    instruction="""
        Analyze assessments from other agents and recommend improvements:
        1. **Functionality & Correctness**: Bugs, error handling, security
        2. **Code Style & Quality**: Readability, maintainability, linting
        3. **Best Practices**: Documentation, configuration files

        Present as clear, actionable Markdown list.
        """,
)

# AGENT 5: Report Generator (Root Agent)
report_generator = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="report_generator",
    description="Compiles comprehensive assessment report",
    tools=[
        AgentTool(description_generator),
        AgentTool(correctness_assessor),
        AgentTool(style_assessor),
        AgentTool(improvement_recommender),
        preload_memory
    ],
    instruction="""
        Compile outputs from all agents into a structured report:

        # Code Assessment Report
        ## 1. Code Description
        ## 2. Code Correctness Assessment
        ## 3. Code Style Assessment
        ## 4. Suggested Improvements

        **CRITICAL**: Provide a complete written text report. 
        Do not just call tools without synthesizing results.
        """,
)

## 6️⃣ Browser Display Utility

This function converts the markdown report to a beautifully styled HTML page with:
- Professional typography and layout
- Markdown rendering via markdown-it library
- Syntax highlighting for code blocks
- Responsive design

In [23]:
def display_report_in_browser(report_text: str, filename="code_assessment_report.html"):
    """Converts markdown report to styled HTML and opens in browser."""
    import webbrowser
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Code Assessment Report</title>
        <style>
            body {{
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
                line-height: 1.6;
                max-width: 900px;
                margin: 0 auto;
                padding: 20px;
                background: #f5f5f5;
            }}
            .container {{
                background: white;
                padding: 40px;
                border-radius: 8px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            h1 {{
                color: #2c3e50;
                border-bottom: 3px solid #3498db;
                padding-bottom: 10px;
            }}
            h2 {{
                color: #34495e;
                margin-top: 30px;
                border-bottom: 2px solid #ecf0f1;
                padding-bottom: 8px;
            }}
            pre {{
                background-color: #f8f9fa;
                border-left: 4px solid #3498db;
                padding: 15px;
                border-radius: 4px;
                overflow-x: auto;
            }}
            code {{
                background-color: #f8f9fa;
                padding: 2px 6px;
                border-radius: 3px;
            }}
        </style>
        <script src="https://cdn.jsdelivr.net/npm/markdown-it@13/dist/markdown-it.min.js"></script>
    </head>
    <body>
        <div class="container">
            <div id="content"></div>
        </div>
        <script>
            var md = window.markdownit();
            var markdownText = `{report_text.replace('`', '\\`').replace('$', '\\$')}`;
            document.getElementById('content').innerHTML = md.render(markdownText);
        </script>
    </body>
    </html>
    """
    
    reports_dir = Path("./reports")
    reports_dir.mkdir(exist_ok=True)
    file_path = reports_dir / filename
    
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    print(f"📄 Report saved to: {file_path}")
    webbrowser.open(f'file://{file_path.absolute()}')
    print(f"🌐 Opening report in browser...")
    
    return str(file_path)

## 7️⃣ Main Execution Function

Configure your code assessment by uncommenting one of the query options:
- **Single file**: Analyze a specific Python file
- **Directory**: Analyze an entire project directory
- **Repository**: Clone and analyze a GitHub repository

The workflow:
1. Initialize ADK environment
2. Create multi-agent application
3. Process assessment request
4. Display results in notebook and browser

In [ ]:
async def main():
    """Main execution function for code assessment workflow."""
    
    # ========== CONFIGURATION ==========
    # Choose ONE of the following options:
    
    code_file_path = "/path/to/your/file.py"
    # code_dir_path = "/path/to/your/directory"
    # repo_http_path = "https://github.com/username/repository"
    
    USER_ID = "explorer"
    session_id = "explorer_session"
    # ===================================

    # Create multi-agent application
    code_broker_app = App(
        name="code_broker",
        root_agent=report_generator,
        resumability_config=ResumabilityConfig(is_resumable=False),
    )

    # Initialize environment
    credentials = await initialize_adk_model()
    session_service = InMemorySessionService()
    memory_service = InMemoryMemoryService()

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            session_id=session_id,
            app_name="code_broker",
            user_id=USER_ID
        )
    except:
        session = await session_service.get_session(
            session_id=session_id,
            app_name="code_broker",
            user_id=USER_ID
        )

    # Create runner
    coding_runner = Runner(
        app=code_broker_app,
        session_service=session_service,
        memory_service=memory_service,
    )
    
    # Construct request (uncomment the one you want to use)
    query = [f"provide a report for the code file:{code_file_path}"]
    # query = [f"provide a report for the directory:{code_dir_path}"]
    # query = [f"provide a report for the repository at:{repo_http_path}"]
    
    user_message = types.Content(role="user", parts=[types.Part(text=query[0])])

    report_text = ""
    
    print("🚀 Starting code assessment...")
    print("⏳ This may take a few minutes...\n")
    
    # Execute agent workflow
    async for event in coding_runner.run_async(
        user_id=USER_ID, session_id=session_id, new_message=user_message
    ):
        if event.is_final_response() and event.content:
            if hasattr(event.content, 'parts') and event.content.parts:
                text_parts = [
                    p.text for p in event.content.parts
                    if hasattr(p, 'text') and p.text
                ]
                if text_parts:
                    report_text = '\n'.join(text_parts)
                    if report_text and report_text != "None":
                        # Display in notebook
                        display(Markdown("## 📊 Code Assessment Report"))
                        display(Markdown("---"))
                        display(Markdown(report_text))
                        
                        # Open in browser
                        print("\n" + "="*60)
                        display_report_in_browser(report_text)
                        print("="*60)
            else:
                print("⚠️  Warning: No text content in response")

    print("\n✅ Assessment complete!")

# Run the assessment
if __name__ == "__main__":
    await main()

/tmp/ipykernel_81052/1332411196.py:19: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=False),


🚀 Starting code assessment...
⏳ This may take a few minutes...



## 📊 Code Assessment Report

---

# Code Assessment Report
## 1. Code Description
The `tools` directory contains the following Python files:

*   `lint_code.py`: This script uses `pylint` to analyze Python code and generate a quality score. It handles file validation, subprocess execution, and score extraction.
*   `read_files.py`: This script provides functions to read individual files (`read_file`) and recursively read all files within a directory and its subdirectories (`read_directory_files`), excluding `.git` directories.
*   `read_github_repository.py`: This script allows cloning a GitHub repository using `git clone`, reading its file contents, and provides a function to clean up temporary directories.

**Functionality Summary:**
*   `lint_code.py`: Analyzes Python code quality using Pylint.
*   `read_files.py`: Reads file and directory contents.
*   `read_github_repository.py`: Clones GitHub repositories and manages temporary directories.

## 2. Code Correctness Assessment

**Functionality (75/100)**:
The code provides functionalities for reading files, directories, and GitHub repositories, as well as linting Python code. Error handling is present, covering cases such as file not found, invalid directory, and failed Git cloning. The `read_directory_files` function correctly skips `.git` directories. However, the linting score extraction seems to be failing, as pylint is running on the directories leading up to the files instead of on the files themselves. Additionally, the `read_file` and `read_directory_files` functions return error messages as strings/dictionaries instead of raising exceptions, which might make them harder to use in a robust system.

**Security (90/100)**:
The code uses `asyncio.create_subprocess_shell`, which can be vulnerable if the command string is constructed from untrusted input. However, in this case, the command strings appear to be constructed safely. The code uses `shutil.rmtree` to remove temporary directories, which can be dangerous if not handled carefully, but here it is being used on directories that were created by the code itself.

**Resource Efficiency (85/100)**:
The `read_github_repository` function clones the entire repository, which might be inefficient for large repositories. Using `--depth 1` helps mitigate this, but a better approach might involve using the GitHub API to selectively download files. The code creates temporary directories which are cleaned up in case of errors.

**Test Coverage (50/100)**:
The provided code does not include any explicit unit tests. Unit tests are crucial for verifying the correctness of the functions, especially the error handling and edge cases. Without tests, it's difficult to assess the reliability of the code.

**Overall Score (75/100)**:
The code provides useful functionalities with reasonable error handling and security considerations. However, the lack of unit tests and the inefficient cloning of GitHub repositories, along with the linting score extraction failure and the use of return strings to signify errors, bring down the overall score.

## 3. Code Style Assessment

**Overall Analysis:**
The file structure appears organized within the `src/tools/` directory. A deeper analysis would require examining the contents of the individual files to evaluate readability, maintainability, and adherence to best practices.

**Readability:** Not assessable without file contents.
**Maintainability:** Not assessable without file contents.
**Linting:** Pylint can provide a quality score based on code style, but an overall score for the directory is not available without running it on individual files.

**Best Practices:**
*   A `README` file explaining the purpose and usage of the tools is recommended.
*   A `.gitignore` file should be present to exclude unnecessary files (e.g., `.pyc` files, virtual environment directories).
*   A `requirements.txt` file is useful to list dependencies.

## 4. Suggested Improvements

Based on the correctness and style assessments, the following improvements are suggested:

**Functionality & Robustness:**
*   **Refine Pylint execution:** Ensure the `lint_code.py` script correctly targets individual Python files for linting rather than their parent directories to obtain accurate scores.
*   **Improve error handling:** Modify `read_file` and `read_directory_files` to raise exceptions (e.g., `FileNotFoundError`, `IOError`) instead of returning error messages as strings or dictionaries. This promotes more robust error propagation and handling in calling code.
*   **Enhance GitHub repository cloning:** For large repositories, consider using the GitHub API to selectively fetch files or implement more granular cloning options beyond `--depth 1` if specific subdirectories are needed, to improve resource efficiency.

**Testing:**
*   **Add unit tests:** Develop comprehensive unit tests for all functions within `lint_code.py`, `read_files.py`, and `read_github_repository.py`. Focus on testing normal execution paths, edge cases, and error conditions (e.g., non-existent files/directories, network failures during cloning).

**Code Style & Project Structure:**
*   **Implement Readme:** Create a `README.md` file at the root of the `src/tools/` directory (or a suitable higher-level directory) to describe the purpose, usage, and functionality of each script.
*   **Add .gitignore:** Include a `.gitignore` file in the project to prevent unnecessary files (like `.pyc` files, `__pycache__` directories, and temporary directories created by `read_github_repository.py`) from being committed to version control.
*   **Specify Dependencies:** Create a `requirements.txt` file to list all necessary Python package dependencies, making it easier for others to set up the development environment.
*   **Adhere to Style Guides:** While a detailed style assessment was not possible, ensure consistent naming conventions, formatting, and commenting practices throughout the codebase. Consider enforcing a style guide (e.g., PEP 8) using tools like `flake8` or configuring `pylint` more strictly.


📄 Report saved to: reports/code_assessment_report.html
🌐 Opening report in browser...

✅ Assessment complete!


Gtk-Message: 16:37:04.043: Failed to load module "xapp-gtk3-module"
Gtk-Message: 16:37:04.043: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
[105416, Main Thread] WARNING: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.: 'glib warning', file /build/firefox/parts/firefox/build/toolkit/xre/nsSigHandlers.cpp:201

(firefox_firefox:105416): Gtk-WARNING **: 16:37:04.128: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.
Gtk-Message: 16:37:04.128: Failed to load module "canberra-gtk-module"
[105416, Main Thread] WARNING: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be l

## 📚 Usage Instructions

### Prerequisites
1. Create a `.env` file in your project root:
   ```
   GOOGLE_API_KEY=your_gemini_api_key_here
   GITHUB_TOKEN=your_github_token_here  # Optional
   ```

2. Install dependencies:
   ```bash
   pip install google-adk python-dotenv pylint
   ```

### Running the Assessment

1. **Configure** the `main()` function with your target:
   - Update `code_file_path`, `code_dir_path`, or `repo_http_path`
   - Uncomment the appropriate `query` line

2. **Execute** all cells from top to bottom

3. **View Results**:
   - Report preview in notebook
   - Full HTML report opens in browser
   - Saved to `./reports/code_assessment_report.html`

### Troubleshooting

- **API Key Error**: Ensure `.env` file exists with valid credentials
- **Pylint Not Found**: Install with `pip install pylint`
- **Rate Limiting**: Retry configuration handles this automatically
- **No Text Response**: Check that agents have proper tool access
- **Linting Unavailable**: Provide the path of the file or directory in quotations


## 8️⃣ Session Memory Storage (Optional)

### Store Assessment Results in Memory

This optional cell demonstrates how to persist the agent session to memory. This allows:
- **Session Persistence**: Save conversation history and agent states
- **Memory Retrieval**: Access previous assessment results
- **Context Preservation**: Maintain context across multiple assessments

**Use Case**: When running multiple assessments, you can reference previous results and compare findings.

**Note**: Uncomment this cell if you want to enable memory storage.

In [25]:
# Optional: Store session to memory for future retrieval
# Uncomment the lines below to enable memory storage

# exploring_session = await session_service.get_session(
#     app_name="code_broker", user_id=USER_ID, session_id=session_id
# )

# await memory_service.add_session_to_memory(exploring_session)
# print("✅ Session stored to memory successfully!")

## 9️⃣ Memory Search and Retrieval (Optional)

### Search Previous Assessment Results

This cell demonstrates the ADK's memory search capability:
- **Semantic Search**: Find relevant information from past assessments
- **Query-Based Retrieval**: Ask questions about previous reports
- **Context Awareness**: Access historical assessment data

**Example Queries:**
- "What are previous reports about?"
- "What were the security findings?"
- "What code style issues were found?"

**Prerequisites**: 
- Session must be stored to memory (see previous cell)
- At least one assessment must have been completed

**Note**: This is useful for tracking improvements over time or comparing different codebases.

In [ ]:
# Optional: Search through stored assessment memories
# Uncomment the lines below to enable memory search

# search_response = await memory_service.search_memory(
#     app_name="code_broker", 
#     user_id=USER_ID, 
#     query="What are previous reports about?"  # Modify this query as needed
# )

# print("🔍 Search Results:")
# print(f"  Found {len(search_response.memories)} relevant memories")
# print()

# for memory in search_response.memories:
#     if memory.content and memory.content.parts:
#         text = memory.content.parts[0].text[:80]
#         print(f"  [{memory.author}]: {text}...")

# # Display full memory content if needed
# if search_response.memories:
#     print("\n📄 Full Memory Content:")
#     for i, memory in enumerate(search_response.memories, 1):
#         if memory.content and memory.content.parts:
#             full_text = memory.content.parts[0].text
#             print(f"\n--- Memory {i} ---")
#             display(Markdown(full_text[:500]))  # Show first 500 chars

Gtk-Message: 16:35:16.480: Failed to load module "xapp-gtk3-module"
Gtk-Message: 16:35:16.480: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
[104782, Main Thread] WARNING: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.: 'glib warning', file /build/firefox/parts/firefox/build/toolkit/xre/nsSigHandlers.cpp:201

(firefox_firefox:104782): Gtk-WARNING **: 16:35:16.560: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.
Gtk-Message: 16:35:16.560: Failed to load module "canberra-gtk-module"
[104782, Main Thread] WARNING: GTK+ module /snap/firefox/7355/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be l

## 🚀 Advanced Features and Next Steps

### Additional Capabilities

#### **1. Batch Processing**
Run assessments on multiple files or repositories:
```python
files = ["file1.py", "file2.py", "file3.py"]
for file_path in files:
    # Run assessment for each file
    # Store results with unique session IDs
```

#### **2. Comparative Analysis**
Compare assessment results across versions:
```python
# Assess version 1
# Store to memory with session_id="v1_assessment"

# Assess version 2
# Store to memory with session_id="v2_assessment"

# Search memory to compare findings
```

#### **3. Custom Agent Configuration**
Modify agent instructions for specialized assessments:
- **Security-focused**: Emphasize vulnerability detection
- **Performance-focused**: Prioritize efficiency metrics
- **Documentation-focused**: Focus on code clarity and comments
- **Research Focused**: Add google search tool to provide more information about the code

#### **4. Export and Share**
- Save reports as PDF using HTML-to-PDF tools
- Export data for analytics dashboards
- Share reports with team members

### Best Practices

1. **Regular Assessments**: Run periodic code assessments during development
2. **Track Improvements**: Use memory to compare current vs. previous scores
3. **Team Integration**: Include reports in code review processes
4. **Continuous Learning**: Adjust agent instructions based on findings

### Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [Gemini API Reference](https://ai.google.dev/docs)
- [Pylint Documentation](https://pylint.pycqa.org/)

---

**Happy Coding! 🎉**